In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Loading HAAR classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
def extractface(input_image):
    ## Return cropped face
    gray = cv2.cvtColor(input_image,cv2.COLOR_BGR2GRAY)
    face = classifier.detectMultiScale(gray,1.3,5)

    if face is ():
        return None
    
    for (x,y,w,h) in face:
        cv2.rectangle(input_image,(x,y),(x+w,y+h),(0,255,0),3)
        #cropped = input_image[y:y+h,x:x+w]
    return cropped

In [12]:
capture_face.release()
cv2.destroyAllWindows()